In [1]:
import os
import random
import shutil

from pathlib import Path
import numpy as np

# import dlib
# import MTCNN

import torch
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader
from facenet_pytorch import MTCNN


# from utils import load_image_and_preprocess

import cv2
import torch
import numpy as np
from pathlib import Path

In [2]:
def load_image_and_preprocess(image_filename, output_image_size, face_detector, face_detector_type="frontal"):
    # read in image with cv2

    image_raw = cv2.imread(image_filename)

    #this might not be needed to correct for color space
#     image = cv2.cvtColor(image_raw, cv2.COLOR_BGR2RGB)
    
    h, w = image.shape[:2]
#     print("H: ", h)
#     print("W: ", w)
    
    if h < 90 or w < 90:
        print("image too small")
        return None
    

    # #use first face detected and check if there were none
    boxes, confidence = face_detector.detect(image_raw)

    if boxes is None:
        return None


    # bounding_box = result[0]["box"]
    bounding_box = boxes[0]
    #collect image height and width
    height, width = image.shape[:2]

    #set scaling factor
    scale=0.98

    #collect coordinates of bounding box
    x1 = bounding_box[0] #dib face.left
    y1 = bounding_box[1] #dib face.top
    x2 = bounding_box[2] #dib face.right
    y2 = bounding_box[3] #dib face.bottom

    #scale bounding box?
    size_bb = int(max(x2 - x1, y2 - y1) * scale)

    #control for out of bounds, x-y top left corner
    center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
    x1 = max(int(center_x - size_bb // 2), 0)
    y1 = max(int(center_y - size_bb // 2), 0)

    # Check for too big bb size for given x, y
    size_bb = min(width - x1, size_bb)
    size_bb = min(height - y1, size_bb)

    # set up for crop with slicing
    cropped_face = image[y1:y1 + size_bb, x1:x1 + size_bb]

    resized_image = cv2.resize(cropped_face, (output_image_size, output_image_size))
    return resized_image

In [11]:
#path to source directory
# src_path = "E:/One Drive/OneDrive/Mainproject/MaskLockData/mask_dataset/face_mask_detection_archive-processed"
# src_path = "E:/One Drive/OneDrive/Mainproject/MaskLockData/mask_dataset/medical_mask_archive-processed"
det_path = "E:/One Drive/OneDrive/Mainproject/MaskLockData/test"

src_path = "E:/One Drive/OneDrive/Mainproject/MaskLockData/data_11_28_2020"

In [13]:
#list the files present
src_file_list = os.listdir(src_path)

print(len(src_file_list))
print(src_file_list[:6])

425
['Screenshot_20201128-142728_Instagram.jpg', 'Screenshot_20201128-142737_Instagram.jpg', 'Screenshot_20201128-142749_Instagram.jpg', 'Screenshot_20201128-142804_Instagram.jpg', 'Screenshot_20201128-142856_Instagram.jpg', 'Screenshot_20201128-142907_Instagram.jpg']


In [26]:
#make the face detector
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device: ", device)

# face_detector = MTCNN(keep_all=True, thresholds=[0.4, 0.6, 0.6], device=device)
# face_detector = MTCNN(keep_all=True, select_largest=False, thresholds=[0.4, 0.6, 0.8], device=device)
face_detector = MTCNN(select_largest=True, thresholds=[0.4, 0.6, 0.8], device=device)

device:  cuda:0


In [27]:
help(MTCNN)


Help on class MTCNN in module facenet_pytorch.models.mtcnn:

class MTCNN(torch.nn.modules.module.Module)
 |  MTCNN(image_size=160, margin=0, min_face_size=20, thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True, select_largest=True, keep_all=False, device=None)
 |  
 |  MTCNN face detection module.
 |  
 |  This class loads pretrained P-, R-, and O-nets and returns images cropped to include the face
 |  only, given raw input images of one of the following types:
 |      - PIL image or list of PIL images
 |      - numpy.ndarray (uint8) representing either a single image (3D) or a batch of images (4D).
 |  Cropped faces can optionally be saved to file
 |  also.
 |  
 |  Keyword Arguments:
 |      image_size {int} -- Output image size in pixels. The image will be square. (default: {160})
 |      margin {int} -- Margin to add to bounding box, in terms of pixels in the final image. 
 |          Note that the application of the margin differs slightly from the davidsandberg/facenet
 

In [28]:
#test a single image
image_size = 224
filename = os.path.join(src_path, src_file_list[0])
img_result = load_image_and_preprocess(filename, image_size, face_detector)

In [29]:
print(img_result)

[[[33 33 33]
  [33 33 33]
  [33 33 33]
  ...
  [ 2  2  2]
  [ 2  2  2]
  [ 2  2  2]]

 [[33 33 33]
  [33 33 33]
  [33 33 33]
  ...
  [ 2  2  2]
  [ 2  2  2]
  [ 2  2  2]]

 [[33 33 33]
  [33 33 33]
  [33 33 33]
  ...
  [ 2  2  2]
  [ 2  2  2]
  [ 2  2  2]]

 ...

 [[80 80 80]
  [80 80 80]
  [80 80 80]
  ...
  [ 1  1  1]
  [ 1  1  1]
  [ 1  1  1]]

 [[80 80 80]
  [80 80 80]
  [80 80 80]
  ...
  [ 1  1  1]
  [ 1  1  1]
  [ 1  1  1]]

 [[80 80 80]
  [80 80 80]
  [80 80 80]
  ...
  [ 1  1  1]
  [ 1  1  1]
  [ 1  1  1]]]


In [30]:
file_count=0
none_count=0
for file in src_file_list:
    filename = os.path.join(src_path, file)
    img_result = load_image_and_preprocess(filename, image_size, face_detector)
    file_count+=1
    if img_result is None:
        none_count+=1
        print("-")
    else:
        cv2.imwrite(os.path.join(det_path, file), img_result)
        print("saving file: ", file)

saving file:  Screenshot_20201128-142728_Instagram.jpg
saving file:  Screenshot_20201128-142737_Instagram.jpg
saving file:  Screenshot_20201128-142749_Instagram.jpg
saving file:  Screenshot_20201128-142804_Instagram.jpg
-
-
saving file:  Screenshot_20201128-142911_Instagram.jpg
-
saving file:  Screenshot_20201128-142919_Instagram.jpg
saving file:  Screenshot_20201128-142923_Instagram.jpg
saving file:  Screenshot_20201128-142929_Instagram.jpg
saving file:  Screenshot_20201128-142938_Instagram.jpg
saving file:  Screenshot_20201128-142943_Instagram.jpg
saving file:  Screenshot_20201128-142948_Instagram.jpg
saving file:  Screenshot_20201128-142953_Instagram.jpg
saving file:  Screenshot_20201128-142957_Instagram.jpg
-
-
saving file:  Screenshot_20201128-143034_Instagram.jpg
saving file:  Screenshot_20201128-143040_Instagram.jpg
-
-
-
-
-
saving file:  Screenshot_20201128-143157_Instagram.jpg
saving file:  Screenshot_20201128-143206_Instagram.jpg
saving file:  Screenshot_20201128-143223_Inst

saving file:  Screenshot_20201128-151648_Instagram.jpg
-
-
-
saving file:  Screenshot_20201128-151721_Instagram.jpg
saving file:  Screenshot_20201128-151727_Instagram.jpg
saving file:  Screenshot_20201128-151733_Instagram.jpg
saving file:  Screenshot_20201128-151741_Instagram.jpg
-
saving file:  Screenshot_20201128-151800_Instagram.jpg
saving file:  Screenshot_20201128-151807_Instagram.jpg
saving file:  Screenshot_20201128-151812_Instagram.jpg
-
saving file:  Screenshot_20201128-151829_Instagram.jpg
saving file:  Screenshot_20201128-151841_Instagram.jpg
-
-
-
saving file:  Screenshot_20201128-151927_Instagram.jpg
saving file:  Screenshot_20201128-151934_Instagram.jpg
saving file:  Screenshot_20201128-151942_Instagram.jpg
-
-
saving file:  Screenshot_20201128-152024_Instagram.jpg
-
-
-
-
-
-
saving file:  Screenshot_20201128-152118_Instagram.jpg
-
-
saving file:  Screenshot_20201128-152142_Instagram.jpg
saving file:  Screenshot_20201128-152147_Instagram.jpg
saving file:  Screenshot_2020

In [31]:
print("number of files processed: ", file_count)
print("number of files with no result: ", none_count)

number of files processed:  425
number of files with no result:  205


In [91]:
# face_detector = MTCNN(keep_all=True, thresholds=[0.4, 0.6, 0.6], device=device)
# dataset: medical_mask_archive-processed
# ~2000 undetected faces, some pretty obvious misses


# face_detector = MTCNN(keep_all=True, select_largest=False, thresholds=[0.6, 0.7, 0.7], device=device)
# number of files processed:  4067
# number of files with no result:  3023
# still some obvious misses

# face_detector = MTCNN(keep_all=True, select_largest=False, thresholds=[0.4, 0.6, 0.8],

# number of files processed:  4067
# number of files with no result:  2988
# a few obvious misses, but less that above!

# face_detector = MTCNN(keep_all=True, select_largest=True, thresholds=[0.4, 0.6, 0.8], device=device)
# number of files processed:  4067
# number of files with no result:  2988


# face_detector = MTCNN(keep_all=True, select_largest=True, thresholds=[0.4, 0.6, 0.8], device=device)
# ALSO including size constraint ov no smaller than 90 px in either direction
# number of files processed:  4067
# number of files with no result:  3027


In [36]:
def prnt(stuff):
    print(stuff)

In [37]:
new_prnt = prnt()

TypeError: prnt() missing 1 required positional argument: 'stuff'

In [ ]:
new_prnt('hello')